In [1]:
import os
from typing import List, Tuple, Dict
from google.generativeai import configure, GenerativeModel
import re
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


  if event.key is 'enter':



In [29]:
def dynamic_game_prompt_template(game_name: str) -> str:
    """Generates a dynamic game-specific prompt where the LLM determines key mistakes and better alternatives, returning a structured JSON output."""

    return (
        f"You are an expert video game coach specializing in analyzing gameplay for {game_name}.\n"
        f"Your task is to analyze a gameplay video and provide **a comprehensive, mistake-focused breakdown** based on the game's mechanics, strategies, and execution.\n\n"

        f"### **Step 1: Identify Key Focus Areas for Analysis**\n"
        f"- Before analyzing the video, list at least **6-8 key factors** that influence success in {game_name}.\n"
        f"- These could include mechanics, strategy, decision-making, positioning, adaptability, execution, etc.\n"
        f"- Weigh their importance before selecting the **4-5 most critical areas** for identifying mistakes.\n\n"

        f"### **Step 2: Extract and List All Mistakes & Better Alternatives**\n"
        f"Provide an exhaustive breakdown of **all major mistakes** made by the player, along with better choices they could have made.\n"
        f"- Each mistake must be accompanied by a **timestamp** and a specific explanation of why it was incorrect.\n"
        f"- Provide **a clearly superior alternative action** with a rationale for why it would have been better.\n\n"

        f"### **Output Format:**\n"
        f"Return the analysis strictly in the following JSON format:\n"
        f"```json\n"
        f"{{\n"
        f"  \"game\": \"{game_name}\",\n"
        f"  \"key_focus_areas\": [\n"
        f"    \"Factor 1\",\n"
        f"    \"Factor 2\",\n"
        f"    \"Factor 3\",\n"
        f"    \"Factor 4\"\n"
        f"  ],\n"
        f"  \"mistakes\": [\n"
        f"    {{\n"
        f"      \"timestamp\": \"00:00:00\",\n"
        f"      \"description\": \"Brief mistake description.\",\n"
        f"      \"why_incorrect\": \"Explanation of why this mistake is bad.\",\n"
        f"      \"better_alternative\": \"What should have been done instead.\",\n"
        f"      \"expected_benefit\": \"Why the alternative is superior.\"\n"
        f"    }}\n"
        f"  ],\n"
        f"  \"repeated_errors\": [\n"
        f"    {{\n"
        f"      \"pattern\": \"Description of recurring mistake.\",\n"
        f"      \"occurrences\": [\"00:01:30\", \"00:04:15\"],\n"
        f"      \"fix\": \"Advice on how to correct this mistake.\"\n"
        f"    }}\n"
        f"  ],\n"
        f"  \"missed_opportunities\": [\n"
        f"    {{\n"
        f"      \"timestamp\": \"00:02:45\",\n"
        f"      \"missed_action\": \"What could have been done instead.\",\n"
        f"      \"expected_outcome\": \"Benefit of the missed opportunity.\"\n"
        f"    }}\n"
        f"  ]\n"
        f"}}\n"
        f"```\n\n"

        f"### **Important Instructions:**\n"
        f"- **Only return JSON output**—do not include any additional text.\n"
        f"- Focus exclusively on **mistakes, missed opportunities, and better alternatives.**\n"
        f"- Do **not** include strengths or positive feedback.\n"
        f"- Always include timestamps when referring to gameplay moments.\n"
        f"- Ensure all explanations are specific, structured, and **actionable**.\n"
        f"- Provide alternatives in a way that makes it clear **how the player should adjust their playstyle.**\n"
        f"- Do not include unnecessary conversational elements—only return the structured JSON output."
    )


In [30]:
prompt=dynamic_game_prompt_template("EA FC 24")

In [4]:


from google.colab import userdata
API_KEY = userdata.get('GOOGLE_API_KEY')
# Example usage:
GAME_NAME = "EA FC 24"


In [5]:
import google.generativeai as genai


genai.configure(api_key=API_KEY)


In [21]:
import os

def list_video_files(directory):
  """Lists all video files in a given directory.

  Args:
    directory: The path to the directory to search.

  Returns:
    A list of strings, where each string is the full path to a video file.
    Returns an empty list if no video files are found or if the directory
    does not exist.
  """

  video_extensions = ['.mp4', '.avi', '.mov', '.mkv', '.wmv', '.flv', '.webm']  # Add more if needed
  video_files = []

  if not os.path.exists(directory):
    print(f"Error: Directory '{directory}' not found.")
    return video_files # Return an empty list

  for filename in os.listdir(directory):
    filepath = os.path.join(directory, filename)
    if os.path.isfile(filepath):  # Make sure it's a file, not a directory
      _, ext = os.path.splitext(filename)  # Split filename into base and extension
      if ext.lower() in video_extensions:
        video_files.append(filepath)

  return video_files


In [23]:
video_dir="/content/videos"
files = list_video_files(video_dir)

In [25]:
video_files=[]
for _file in files:

  print(f"Uploading file...")
  video_file = genai.upload_file(path=_file)
  print(f"Completed upload: {video_file.uri}")
  video_files.append(video_file)

Uploading file...
Completed upload: https://generativelanguage.googleapis.com/v1beta/files/gw5j02elk7ni
Uploading file...
Completed upload: https://generativelanguage.googleapis.com/v1beta/files/b9qgqqwdm3de


In [32]:
# Set the model to Gemini Flash.
model = genai.GenerativeModel(model_name="models/gemini-2.0-flash")
output=[]
for _video in video_files:
  # Make the LLM request.
  print("Making LLM inference request...")
  response = model.generate_content([prompt, _video],
                                    request_options={"timeout": 600})

  output.append({
      "video": _video,
      "response": response.text,
  })

Making LLM inference request...
Making LLM inference request...


In [41]:
help(response)

Help on GenerateContentResponse in module google.generativeai.types.generation_types object:

class GenerateContentResponse(BaseGenerateContentResponse)
 |  GenerateContentResponse(done: 'bool', iterator: 'None | Iterable[protos.GenerateContentResponse] | AsyncIterable[protos.GenerateContentResponse]', result: 'protos.GenerateContentResponse', chunks: 'Iterable[protos.GenerateContentResponse] | None' = None)
 |  
 |  Instances of this class manage the response of the `generate_content` method.
 |  
 |  These are returned by `GenerativeModel.generate_content` and `ChatSession.send_message`.
 |  This object is based on the low level `protos.GenerateContentResponse` class which just has `prompt_feedback`
 |  and `candidates` attributes. This class adds several quick accessors for common use cases.
 |  
 |  The same object type is returned for both `stream=True/False`.
 |  
 |  ### Streaming
 |  
 |  When you pass `stream=True` to `GenerativeModel.generate_content` or `ChatSession.send_mes

In [51]:
response.text

'```json\n{\n  "game": "EA FC 24",\n  "key_focus_areas": [\n    "Passing Accuracy & Decision-Making",\n    "Defensive Positioning & Awareness",\n    "Use of Skill Moves & Dribbling",\n    "Shot Selection & Timing"\n  ],\n  "mistakes": [\n    {\n      "timestamp": "00:28",\n      "description": "Unnecessary risk when passing the ball to a player who is under pressure.",\n      "why_incorrect": "Passing to a player marked by multiple opponents significantly increases the chance of losing possession. The passing lane is tight and the margin for error is very small.",\n      "better_alternative": "Pass the ball backwards to a defender or midfielder who is in a better position and can see the passing options more clearly.",\n      "expected_benefit": "Increases the likelihood of maintaining possession and building a more controlled attack."\n    },\n    {\n      "timestamp": "00:51",\n      "description": "Trying to dribble through many players instead of passing.",\n      "why_incorrect": 

In [54]:
import json
def extract_json(response: str):
    """Extracts the first valid JSON object from the response text."""
    match = re.search(r'\{.*\}', response, re.DOTALL)
    if match:
        json_text = match.group(0)  # Extracts only the JSON portion
        return json.loads(json_text)
    raise ValueError("No valid JSON found")


In [38]:
output[0]["response"]

'```json\n{\n  "game": "EA FC 24",\n  "key_focus_areas": [\n    "Defensive Positioning & Awareness",\n    "Passing Accuracy & Decision-Making",\n    "Shot Selection & Timing",\n    "Player Switching & Control",\n    "Set Piece Execution"\n  ],\n  "mistakes": [\n    {\n      "timestamp": "00:55:55",\n      "description": "Weak penalty kick.",\n      "why_incorrect": "The shot lacked power and placement, making it an easy save for the goalkeeper.",\n      "better_alternative": "Use timed finishing to increase the power and accuracy of the shot. Aim for the top corner or a side with power.",\n      "expected_benefit": "Increased chance of scoring due to a more powerful and precise shot."\n    },\n    {\n      "timestamp": "01:52",\n      "description": "Another penalty kick with poor placement.",\n      "why_incorrect": "Shooting straight down the middle is a low-percentage choice against human players who anticipate the direction.",\n      "better_alternative": "Aim the penalty towards e

In [63]:
out["response"]

'```json\n{\n  "game": "EA FC 24",\n  "key_focus_areas": [\n    "Defensive Positioning & Awareness",\n    "Passing Accuracy & Decision-Making",\n    "Shot Selection & Timing",\n    "Player Switching & Control",\n    "Set Piece Execution"\n  ],\n  "mistakes": [\n    {\n      "timestamp": "00:55:55",\n      "description": "Weak penalty kick.",\n      "why_incorrect": "The shot lacked power and placement, making it an easy save for the goalkeeper.",\n      "better_alternative": "Use timed finishing to increase the power and accuracy of the shot. Aim for the top corner or a side with power.",\n      "expected_benefit": "Increased chance of scoring due to a more powerful and precise shot."\n    },\n    {\n      "timestamp": "01:52",\n      "description": "Another penalty kick with poor placement.",\n      "why_incorrect": "Shooting straight down the middle is a low-percentage choice against human players who anticipate the direction.",\n      "better_alternative": "Aim the penalty towards e

In [65]:
json_data

{'game': 'EA FC 24',
 'key_focus_areas': ['Passing Accuracy & Decision-Making',
  'Defensive Positioning & Awareness',
  'Use of Skill Moves & Dribbling',
  'Shot Selection & Timing'],
 'mistakes': [{'timestamp': '00:28',
   'description': 'Unnecessary risk when passing the ball to a player who is under pressure.',
   'why_incorrect': 'Passing to a player marked by multiple opponents significantly increases the chance of losing possession. The passing lane is tight and the margin for error is very small.',
   'better_alternative': 'Pass the ball backwards to a defender or midfielder who is in a better position and can see the passing options more clearly.',
   'expected_benefit': 'Increases the likelihood of maintaining possession and building a more controlled attack.'},
  {'timestamp': '00:51',
   'description': 'Trying to dribble through many players instead of passing.',
   'why_incorrect': 'Attempting to dribble through a cluster of defenders rarely succeeds due to tight marking a

In [66]:

def format_analysis(json_data):
    """Formats the JSON data into a human-readable string.

    Args:
        json_data (dict): The JSON data representing the game analysis.

    Returns:
        str: A formatted string for user readability.
    """

    output = f"Game: {json_data['game']}\n\n"
    output += "Key Focus Areas:\n"
    for area in json_data['key_focus_areas']:
        output += f"- {area}\n"

    output += "\nMistakes:\n"
    for mistake in json_data['mistakes']:
        output += f"  Timestamp: {mistake['timestamp']}\n"
        output += f"  Description: {mistake['description']}\n"
        output += f"  Why Incorrect: {mistake['why_incorrect']}\n"
        output += f"  Better Alternative: {mistake['better_alternative']}\n"
        output += f"  Expected Benefit: {mistake['expected_benefit']}\n"
        output += "\n"

    output += "Repeated Errors:\n"
    for error in json_data['repeated_errors']:
        output += f"  Pattern: {error['pattern']}\n"
        output += f"  Occurrences: {', '.join(error['occurrences'])}\n"
        output += f"  Fix: {error['fix']}\n"
        output += "\n"

    output += "Missed Opportunities:\n"
    for opportunity in json_data['missed_opportunities']:
        output += f"  Timestamp: {opportunity['timestamp']}\n"
        output += f"  Missed Action: {opportunity['missed_action']}\n"
        output += f"  Expected Outcome: {opportunity['expected_outcome']}\n"
        output += "\n"

    return output

Game: EA FC 24

Key Focus Areas:
- Passing Accuracy & Decision-Making
- Defensive Positioning & Awareness
- Use of Skill Moves & Dribbling
- Shot Selection & Timing

Mistakes:
  Timestamp: 00:28
  Description: Unnecessary risk when passing the ball to a player who is under pressure.
  Why Incorrect: Passing to a player marked by multiple opponents significantly increases the chance of losing possession. The passing lane is tight and the margin for error is very small.
  Better Alternative: Pass the ball backwards to a defender or midfielder who is in a better position and can see the passing options more clearly.
  Expected Benefit: Increases the likelihood of maintaining possession and building a more controlled attack.

  Timestamp: 00:51
  Description: Trying to dribble through many players instead of passing.
  Why Incorrect: Attempting to dribble through a cluster of defenders rarely succeeds due to tight marking and increased chances of losing the ball. The area is too congested 

In [69]:
for out in output:
  print(f"Video: {out['video'].display_name}")

  json_data = extract_json(out["response"])

  print(format_analysis(json_data))

Video: videoplayback (2)111.mp4
Game: EA FC 24

Key Focus Areas:
- Defensive Positioning & Awareness
- Passing Accuracy & Decision-Making
- Shot Selection & Timing
- Player Switching & Control
- Set Piece Execution

Mistakes:
  Timestamp: 00:55:55
  Description: Weak penalty kick.
  Why Incorrect: The shot lacked power and placement, making it an easy save for the goalkeeper.
  Better Alternative: Use timed finishing to increase the power and accuracy of the shot. Aim for the top corner or a side with power.
  Expected Benefit: Increased chance of scoring due to a more powerful and precise shot.

  Timestamp: 01:52
  Description: Another penalty kick with poor placement.
  Why Incorrect: Shooting straight down the middle is a low-percentage choice against human players who anticipate the direction.
  Better Alternative: Aim the penalty towards either the left or right side of the goal, utilizing the timed finishing mechanic.
  Expected Benefit: Forces the keeper to commit and cover mor